In [107]:
import sys
import numpy as np
import cv2
from itertools import groupby

image_count = 612

In [110]:
def map_binary(mask):
    (threshold, black_and_white) = cv2.threshold(mask, 127, 255, cv2.THRESH_BINARY)
    return black_and_white

def rle(binary_mask):
    rle = {'counts': [], 'size': list(binary_mask.shape)}
    counts = rle.get('counts')
    for i, (value, elements) in enumerate(groupby(binary_mask.ravel(order='F'))):
        if i == 0 and value == 1:
            counts.append(0)
        counts.append(len(list(elements)))
    return rle

In [111]:
img = cv2.imread('data/3.tif', 0)
black_and_white = map_binary(img)
rle = rle(black_and_white)

print(rle)

{'counts': [11650, 105, 183, 6, 6, 1, 21, 4, 4, 64, 182, 107, 181, 108, 181, 108, 180, 109, 180, 109, 180, 109, 179, 110, 179, 110, 178, 111, 177, 112, 177, 112, 177, 112, 176, 113, 176, 113, 175, 114, 175, 114, 175, 114, 174, 115, 174, 115, 174, 115, 173, 116, 173, 115, 174, 115, 173, 116, 173, 116, 173, 116, 173, 116, 173, 116, 173, 116, 173, 116, 173, 116, 174, 115, 174, 115, 174, 115, 174, 115, 173, 116, 172, 117, 171, 118, 170, 119, 169, 120, 168, 121, 167, 122, 165, 124, 164, 125, 162, 127, 161, 127, 161, 127, 160, 128, 160, 128, 160, 128, 160, 128, 159, 129, 159, 129, 159, 129, 159, 129, 159, 129, 159, 129, 158, 130, 158, 130, 158, 130, 158, 130, 158, 130, 158, 130, 158, 130, 158, 130, 158, 130, 158, 130, 158, 130, 158, 130, 158, 130, 158, 130, 159, 129, 159, 129, 159, 129, 159, 129, 160, 128, 160, 128, 160, 128, 161, 127, 161, 127, 162, 126, 162, 126, 163, 125, 163, 125, 164, 124, 164, 124, 165, 123, 167, 121, 167, 121, 168, 120, 169, 119, 171, 117, 172, 116, 173, 115, 175, 113